In [130]:
#for Netcdf manipulation
import xarray as xr
from netCDF4 import Dataset
import netCDF4

#for array manipulation
import numpy as np
import pandas as pd

from scipy import stats

In [163]:
# read .nc file
dataDIR = 'data/EmEm.monthly.1981-2004.nc'
Data = xr.open_dataset(dataDIR, decode_times=False)
Data

<xarray.Dataset>
Dimensions:  (time: 288, lon: 200, lat: 100, plev: 4)
Coordinates:
  * time     (time) float64 0.0 1.0 2.0 3.0 4.0 ... 284.0 285.0 286.0 287.0
  * lon      (lon) float32 -99.88 -99.62 -99.38 -99.12 ... -50.62 -50.38 -50.12
  * lat      (lat) float32 5.125 5.375 5.625 5.875 ... 29.12 29.38 29.62 29.88
  * plev     (plev) float32 950.0 850.0 700.0 500.0
Data variables:
    t        (time, plev, lat, lon) float64 ...
    q        (time, plev, lat, lon) float64 ...
    r        (time, plev, lat, lon) float64 ...
Attributes: (12/99)
    CDI:                                             Climate Data Interface v...
    Conventions:                                     CF-1.4
    source:                                          RegCM Model output file
    institution:                                     ICTP
    history:                                         Thu Nov 17 11:43:21 2022...
    title:                                           ICTP Regional Climatic m...
    ...                                              ...
    holtslag_critical_land_richardson:               0.25
    holtslag_zhnew_factor:                           0.25
    holtslag_th10_estimate:                          1
    holtslag_th10_maximize:                          1
    frequency:                                       mon
    CDO:                                             Climate Data Operators v...

In [176]:
#variable (air temperature)
dataTemp = Data['t']

#selecting a layer (no empty values)
dataTemp = dataTemp.sel(lon=np.arange(-90,-55,0.25),lat=np.arange(12,30,0.25), plev=500, method='nearest').dropna(dim='time')
dataDIR = 'airTemperatureLv500.nc'
dataTemp.to_netcdf(dataDIR)

#new layer
air_temperature = xr.open_dataset(dataDIR, decode_times=False)

#preliminary overview
minAT = air_temperature.min()
maxAT = air_temperature.max()
meanAT = air_temperature.mean()
stdAT = air_temperature.std()

#fixed threshold (over dataset mean)
thresholdAT = air_temperature.where(air_temperature.time > meanAT).dropna(dim='time')

#new product
dataDIR = 'over.mean.airTemperatureLv500.nc'
thresholdAT.to_netcdf(dataDIR)
thresholdAT

<xarray.Dataset>
Dimensions:  (time: 22, lat: 72, lon: 140)
Coordinates:
  * time     (time) float64 266.0 267.0 268.0 269.0 ... 284.0 285.0 286.0 287.0
  * lon      (lon) float32 -89.88 -89.62 -89.38 -89.12 ... -55.62 -55.38 -55.12
  * lat      (lat) float32 12.12 12.38 12.62 12.88 ... 29.12 29.38 29.62 29.88
    plev     float32 500.0
Data variables:
    t        (time, lat, lon) float64 268.7 268.7 268.6 ... 261.6 261.7 261.7

In [177]:
#variable (specific humidity)
dataTemp = Data['q']

#selecting a layer (no empty values)
dataTemp = dataTemp.sel(lon=np.arange(-90,-55,0.25),lat=np.arange(12,30,0.25), plev=500, method='nearest').dropna(dim='time')
dataDIR = 'specificHumidityLv500.nc'
dataTemp.to_netcdf(dataDIR)

#new layer
specific_humidity = xr.open_dataset(dataDIR, decode_times=False)

In [178]:
#variable (relative humidity)
dataTemp = Data['r']

#selecting a layer (no empty values)
dataTemp = dataTemp.sel(lon=np.arange(-90,-55,0.25),lat=np.arange(12,30,0.25), plev=500, method='nearest').dropna(dim='time')
dataDIR = 'relativeHumidityLv500.nc'
dataTemp.to_netcdf(dataDIR)

#new layer
relative_humidity = xr.open_dataset(dataDIR, decode_times=False)

In [206]:
#np arrays datasets (same size)
np_specific_humidity = np.array(specific_humidity['q'])
np_relative_humidity = np.array(relative_humidity['r'])

#np array (dataset) shape
np_shape = np_specific_humidity.shape

output = []
correl = -9999

#Spearman correlation between two datasets for each grid point (lat, lon)
for i in np.arange(np_shape[1]):
    for j in np.arange(np_shape[2]):
        
        spearmancor, pval = stats.spearmanr(np_specific_humidity[:,i,j], np_relative_humidity[:,i,j])
        #print(f'spearman correlation: {spearmancor} and p-value: {pval}')
        
        if pval > 0.05:
           correl = -9999
        else:
           correl = spearmancor
        
        output.append(correl)


print(output)

10080
(288, 72, 140)
[0.9858607718553707, 0.986593202560795, 0.9872713791398913, 0.9874532309336935, 0.988169586342428, 0.9881163369221434, 0.9880228992601346, 0.9880846886817857, 0.9879987863151002, 0.9879530721901386, 0.987913386301436, 0.9880113451406389, 0.987595396838793, 0.986715776698054, 0.9857618083101247, 0.9850213399563558, 0.9845003998730053, 0.9837011562157144, 0.9837021609217577, 0.983378143222856, 0.982808474896415, 0.9829968572794973, 0.9833670914563819, 0.9843326139638067, 0.9846053916545098, 0.9850665517282954, 0.9855699094558915, 0.9870945508763046, 0.9876245333140431, 0.9878983157107893, 0.9885689569945626, 0.9891230523773357, 0.9895405077382461, 0.9897881677778716, 0.9904939737731536, 0.9909370491381633, 0.9911294503454181, 0.9914037350951861, 0.991655413958984, 0.9918342516346567, 0.992338614068296, 0.992811830614599, 0.9930243259427157, 0.9933880295303201, 0.9936693472223898, 0.993865767253817, 0.9937517331179246, 0.9938768190202912, 0.9941284978840892, 0.9940938